In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from pathlib import Path
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
)

In [ ]:
train_generator = datagen.flow_from_directory(
    'Image_processing/DATASET/TRAIN',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
model = load_model('model.h5')
feature_extractor = tf.keras.Model(inputs= model.input, outputs= model.layers[-2].output)
test_generator = test_datagen.flow_from_directory(
    'Image_processing/DATASET/TEST',
    target_size=(128, 128),
    class_mode='binary',
    shuffle=False
)

In [ ]:
X_train = feature_extractor.predict(train_generator)

In [ ]:
X_test = feature_extractor.predict(test_generator)

In [ ]:
labels = []
for i in tqdm(range(706)):
    inputs, label_batch = next(train_generator)
    labels.extend(label_batch)
labels = np.array(labels)
labels.shape

In [ ]:
y_test = []
for i in tqdm(range(79)):
    inputs, label_batch = next(test_generator)
    y_test.extend(label_batch)
y_test = np.array(y_test)
y_test.shape

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
svm = SVC(kernel='linear', C=1, gamma='auto', verbose=True)
svm.fit(X_train_scaled, labels)

In [ ]:
y_pred = svm.predict(X_train[:2000])

In [ ]:
score = accuracy_score(labels[:2000], y_pred)
score